In [1]:
import requests
import json
import time
import chromadb
from sentence_transformers import SentenceTransformer
from unstructured.partition.html import partition_html
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from newspaper import Article

# 🔹 CONFIGURACIÓN: Define la API de Mediastack y el número de artículos a obtener
API_KEY = "356bb7cd80f02083d604ba6ba1dfadd8"
MAX_ARTICLES = 10  # Puedes cambiar este valor a 5, 10, etc.

# Dominio de Mediastack
BASE_URL = f"http://api.mediastack.com/v1/news?access_key={API_KEY}&countries=us&limit={MAX_ARTICLES}"

# Lista de dominios de pago (para evitar scrapeo de contenido restringido)
paywalled_domains = ["nytimes.com", "washingtonpost.com", "theatlantic.com", "bloomberg.com"]

# User-Agent Rotator
ua = UserAgent()

def is_paywalled(url):
    """Verifica si un artículo proviene de un dominio de pago."""
    return any(domain in url for domain in paywalled_domains)

def extract_full_text(url):
    """Extrae el texto completo de un artículo usando newspaper3k, Unstructured y BeautifulSoup."""
    try:
        headers = {'User-Agent': ua.random}
        page = requests.get(url, headers=headers, timeout=10)

        if page.status_code != 200:
            return f"Error: Página respondió con código {page.status_code}"

        # Intento 1: newspaper3k (mejor extracción de texto)
        article = Article(url)
        article.download()
        article.parse()
        if len(article.text) > 500:
            return article.text

        # Intento 2: Unstructured (fallback)
        elements = partition_html(text=page.text)
        extracted_text = "\n".join([el.text for el in elements if el.text.strip()])
        if len(extracted_text) > 500:
            return extracted_text

        # Intento 3: BeautifulSoup (última opción)
        soup = BeautifulSoup(page.text, "html.parser")
        paragraphs = soup.find_all("p")
        extracted_text = "\n".join([p.get_text() for p in paragraphs])
        return extracted_text if len(extracted_text) > 500 else "No se pudo extraer contenido."

    except Exception as e:
        return f"Error extrayendo contenido: {str(e)}"

# 🔹 Obtener noticias desde Mediastack
response = requests.get(BASE_URL)
news_data = response.json().get("data", [])[:5]  # Límite de artículos

articles_list = []

# 🔹 Procesar cada noticia
for i, article in enumerate(news_data):
    url = article.get("url", "")

    if not url or is_paywalled(url):
        print(f"🚫 Saltando artículo restringido: {url}")
        continue

    print(f"🔍 [{i+1}/{MAX_ARTICLES}] Procesando: {url}")
    full_text = extract_full_text(url)

    articles_list.append({
        "title": article.get("title", "Título desconocido"),
        "url": url,
        "content": full_text
    })

    time.sleep(2)  # Evitar bloqueos por exceso de requests

# 🔹 Guardar artículos en JSON
with open("news.json", "w", encoding="utf-8") as f:
    json.dump(articles_list, f, indent=4)

print(f"✅ Noticias guardadas en 'news.json'.")

# 🔹 INTEGRACIÓN CON CHROMADB (Embeddings)
print("🔄 Convirtiendo artículos en embeddings y almacenándolos en ChromaDB...")

# Inicializar ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("news_articles")

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Convertir cada artículo en embeddings y guardarlos en ChromaDB
for article in articles_list:
    text = article["title"] + " " + article["content"]
    embedding = embedding_model.encode(text).tolist()

    collection.add(
        ids=[article["url"]],
        embeddings=[embedding],
        metadatas=[{"title": article["title"], "url": article["url"]}],
        documents=[text]
    )

print("✅ Artículos convertidos en embeddings y almacenados en ChromaDB.")


ModuleNotFoundError: No module named 'fake_useragent'